# Metrics Heatmap

<a href="https://colab.research.google.com/github/netdata/netdata-community/blob/main/netdata-agent-api/netdata-pandas/metrics_heatmap.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this notebook we will use the [netdata-pandas](https://github.com/netdata/netdata-pandas) Python package to pull some data from some demo Netdata servers and make some pretty looking heatmaps, because we all love a good heatmap don't we. 

**Note**: you can click the "Open in Colab" button above to open this notebook in [Google Colab](https://colab.research.google.com/notebooks/intro.ipynb#recent=true) where you can just get going with it without having to set up python enviornments or any messy stuff like that.

In [1]:
# if you need to, uncomment below to install netdata-pandas and seaborn packages
#!pip install plotly==4.14.1 netdata-pandas==0.0.33

In [2]:
import numpy as np
import pandas as pd
from netdata_pandas.data import get_data
import plotly.express as px

Lets pull some data for the last 15 minutes.

In [3]:
# inputs
hosts = ['london.my-netdata.io']
charts_regex = 'system.*'
before = 0
after = -60*15
resample_freq = '10s'

In [4]:
# get the data
df_raw = get_data(hosts=hosts, charts_regex=charts_regex, after=after, before=before, index_as_datetime=True)
print(df_raw.shape)
df_raw.head()

(902, 72)


,system.active_processes|active,system.cpu|guest,system.cpu|guest_nice,system.cpu|iowait,system.cpu|irq,system.cpu|nice,system.cpu|softirq,system.cpu|steal,system.cpu|system,system.cpu|user,...,system.softirqs|RCU,system.softirqs|SCHED,system.softirqs|TASKLET,system.softirqs|TIMER,system.softnet_stat|dropped,system.softnet_stat|flow_limit_count,system.softnet_stat|processed,system.softnet_stat|received_rps,system.softnet_stat|squeezed,system.uptime|uptime
time_idx,,,,,,,,,,,,,,,,,,,,,
2020-12-17 17:26:35,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-12-17 17:26:36,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-12-17 17:26:37,210.0,0.0,0.0,0.0,0.0,0.0,0.000000,1.496259,1.745636,1.246883,...,122.47433,115.72116,34.32288,155.37008,0.0,0.0,95.88854,0.0,0.0,67362000.0
2020-12-17 17:26:38,210.0,0.0,0.0,0.0,0.0,0.0,0.247525,0.990099,1.237624,0.990099,...,107.78124,98.12782,26.59372,135.10801,0.0,0.0,75.87651,0.0,0.0,67362000.0
2020-12-17 17:26:39,210.0,0.0,0.0,0.0,0.0,0.0,0.246305,2.463054,1.231527,0.492611,...,108.43762,109.53677,29.73058,146.88052,0.0,0.0,91.85455,0.0,0.0,67362000.0


## Heatmaps!

In [5]:
# lets resample to 5 sec frequency
df = df_raw.resample(resample_freq).mean()

# lets min-max normalize our data so metrics can be compared on a heatmap
df=(df-df.min())/(df.max()-df.min())

# drop na cols
df = df.dropna(how='all', axis=1)

# lets sort cols by their std to try make heatmap prettier
df = df[df.std().sort_values().index]

print(df.shape)
df.head(10)

(91, 48)


,system.interrupts|virtio0-output.0_26,system.ram|buffers,system.active_processes|active,system.processes|blocked,system.ipv6|sent,system.interrupts|MCP,system.softirqs|NET_TX,system.net|sent,system.ip|sent,system.net|received,...,system.interrupts|CAL,system.ram|used,system.entropy|entropy,system.interrupts|TLB,system.cpu|steal,system.ram|free,system.load|load5,system.uptime|uptime,system.ram|cached,system.ipv6|received
time_idx,,,,,,,,,,,,,,,,,,,,,
2020-12-17 17:26:30,0.0,0.5,0.4375,0.0,1.0,0.0,0.417080,0.689316,0.660478,0.627536,...,0.385822,0.634166,0.294540,0.343424,0.259126,0.508540,0.038462,0.000000,0.000000,0.904634
2020-12-17 17:26:40,0.0,0.0,0.5000,1.0,1.0,0.0,0.210803,0.824426,0.794327,0.247729,...,0.621637,0.477245,0.313937,0.623194,0.430617,0.656540,0.038462,0.010016,0.003880,0.227855
2020-12-17 17:26:50,0.0,0.0,0.4375,1.0,1.0,0.0,0.232981,0.706939,0.681465,0.218097,...,0.459021,0.533632,0.341954,0.451003,0.330621,0.600215,0.096154,0.021145,0.016584,0.051120
2020-12-17 17:27:00,0.0,0.0,0.4375,1.0,1.0,0.0,0.098766,0.845960,0.778162,0.387852,...,0.442505,0.592661,0.370690,0.507033,0.360764,0.539987,0.151716,0.032273,0.035686,0.260481
2020-12-17 17:27:10,0.0,0.0,0.5000,0.7,1.0,0.0,0.125668,0.842390,0.828669,0.159896,...,0.731328,0.531692,0.399066,0.722090,0.283508,0.596390,0.115385,0.043402,0.041994,0.695309
2020-12-17 17:27:20,0.0,0.0,0.4375,1.0,1.0,0.0,0.071863,0.924956,0.882342,0.267424,...,0.964609,0.435244,0.428161,0.936633,0.000000,0.685929,0.115385,0.054531,0.051589,0.509778
2020-12-17 17:27:30,0.0,0.0,0.4375,1.0,1.0,0.0,0.112217,0.839305,0.797493,0.262123,...,0.489070,0.451343,0.456537,0.501504,0.057816,0.666641,0.115385,0.065660,0.069980,0.090523
2020-12-17 17:27:40,0.0,0.0,0.4375,1.0,1.0,0.0,0.022729,0.888095,0.823070,0.375662,...,0.762961,0.475388,0.486351,0.764439,0.156408,0.640624,0.096154,0.076789,0.084106,0.000000
2020-12-17 17:27:50,0.0,0.0,0.4375,1.0,1.0,0.0,0.295864,0.666910,0.651226,0.154427,...,0.483171,0.560215,0.513649,0.462780,0.218615,0.558201,0.076923,0.087917,0.092990,0.017774


In [6]:
fig = px.imshow(df.transpose())
fig.update_layout(
    autosize=False,
    width=1000,
    height=1200)
fig.show()